In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
headers = {"user-agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}

In [3]:
#print(clothesInfoList)
def get_all_pages(headers):

  product_links = []

  for page in range(1,4):
    response = requests.get(f"https://zyga.fr/115-toute-la-collection-2023?page={page}", headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    clothesInfoList = soup.find_all("div", class_="products row")

    for item in clothesInfoList:
      # Retrieve the clothes link
      for link in item.find_all("a", href=True):
        #print(link.get('href'))
        # print(link["href"])
        product_links.append(link["href"])
  # print(clothesLinks)

  charToDelete = '#'
  product_links = [character for character in product_links if character != charToDelete]
  return product_links

In [4]:
product_links = get_all_pages(headers)

In [5]:
print(len(product_links))
print(product_links)

216
['https://zyga.fr/Robes-femme-lin-printemps-ete/2372-10492-robe-ample-midi-summer-gaze-lin-vert-green.html#/143-taille-34', 'https://zyga.fr/Robes-femme-lin-printemps-ete/2372-10492-robe-ample-midi-summer-gaze-lin-vert-green.html#/143-taille-34', 'https://zyga.fr/Chemises-tops-femme-lin-printemps-ete/2429-10779-blouse-fronce-imprimee-france-gaze-lin-solange-vert-green.html#/143-taille-34', 'https://zyga.fr/Chemises-tops-femme-lin-printemps-ete/2429-10779-blouse-fronce-imprimee-france-gaze-lin-solange-vert-green.html#/143-taille-34', 'https://zyga.fr/Robes-femme-lin-printemps-ete/2459-10948-robe-polo-evasee-selma-lin-leger-vert-green.html#/143-taille-34', 'https://zyga.fr/Robes-femme-lin-printemps-ete/2459-10948-robe-polo-evasee-selma-lin-leger-vert-green.html#/143-taille-34', 'https://zyga.fr/Robes-femme-lin-printemps-ete/2431-10793-robe-midi-sans-manches-marie-lin-leger-imprime-vert-green.html#/141-taille-s', 'https://zyga.fr/Robes-femme-lin-printemps-ete/2431-10793-robe-midi-sans

In [6]:
def retrieve_product_info(product_links, headers):
  product_list = []
  for link in product_links:

    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    charToDelete = '\n'
    name = soup.find("h1", class_="product-title--big").text
    # print("clothes_name : "+ name + "\n")

    price = soup.find("div", class_="current-price").text
    # print(price[1:7] + "€")
    # print("price : "+ price + "\n")

    description = soup.find("div", itemprop="description").text
    description_modified = [character for character in description if character != charToDelete]

    # print(type(description_modified))
    # print("description : "+ description + "\n")

    fabric = []
    for elt in soup.find("div", class_="rte"):
      for s in elt:
        # print(s)
        fabric.append(s)

    # list_of_price = [s.strip('\xa0€') for s in fabric]

    
    fabric_lin = fabric[1]
    fabric_washing_advice = fabric[15:20]
    # print("fabric : ", fabric_lin, "\n")
    # print("fabric : ", fabric_washing_advice, "\n")
    fabric_washing_advice = [character for character in fabric_washing_advice if character != charToDelete]

    # print(type(fabric))
    zyga_clothes = {
        "name" : name,
        "price" : price[1:7] + "€",
        "description" : description,
        "fabric" : fabric_lin,
        "washing advice" : fabric_washing_advice, 
        "url" : link
    }

    product_list.append(zyga_clothes)
    # print("Saving", description_modified["description"])
  return product_list

In [7]:
product_list = retrieve_product_info(product_links, headers)
print(len(product_list))

216


In [8]:
def product_df(product_list):
  df_zyga_clothes = pd.DataFrame(product_list)
  return df_zyga_clothes


In [9]:
df_zyga_clothes = product_df(product_list)

In [10]:
df_zyga_clothes.head()

,name,price,description,fabric,washing advice,url
0,Robe midi ample SUMMER en Lin - Green,"178,00€",\nRobe midi ample en gaze de Lin - Vert Green\...,100% Gaze de Lin (85g/m2),"[- Lavage en machine, 40° maximum avec un esso...",https://zyga.fr/Robes-femme-lin-printemps-ete/...
1,Robe midi ample SUMMER en Lin - Green,"178,00€",\nRobe midi ample en gaze de Lin - Vert Green\...,100% Gaze de Lin (85g/m2),"[- Lavage en machine, 40° maximum avec un esso...",https://zyga.fr/Robes-femme-lin-printemps-ete/...
2,Blouse à fronces SOLANGE en Lin imprimé - Green,"168,00€",\nBlouse froncée en gaze de Lin lavée - Vert G...,100% Gaze de Lin (85g/m2) imprimée et lavée,[- Laver en machine maximum 40° avec un essora...,https://zyga.fr/Chemises-tops-femme-lin-printe...
3,Blouse à fronces SOLANGE en Lin imprimé - Green,"168,00€",\nBlouse froncée en gaze de Lin lavée - Vert G...,100% Gaze de Lin (85g/m2) imprimée et lavée,[- Laver en machine maximum 40° avec un essora...,https://zyga.fr/Chemises-tops-femme-lin-printe...
4,Robe polo SELMA en Lin - Green,"178,00€",\nRobe col polo évasée en Lin Léger lavé - Ver...,100% Lin Léger (135g/m²),"[Entretien :, - Lavage en machine à 40° maximu...",https://zyga.fr/Robes-femme-lin-printemps-ete/...


In [11]:
df_zyga_clothes.shape

(216, 6)

In [12]:
df_zyga_clothes.to_csv('zyga_data.csv', encoding='utf-8')